In [ ]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
unique_values_per_column = application_df.nunique()
unique_values_per_column

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at ORGANIZATION value counts for binning
org_vc=application_df['AFFILIATION'].value_counts()
org_vc

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500

application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts[classification_counts > 1]
classification_counts_gt1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff_value = 100
classifications_to_replace = classification_counts[classification_counts < classification_cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
target = application_df_encoded['IS_SUCCESSFUL']
features = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, stratify=target)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (25724, 49)
X_test shape: (8575, 49)
y_train shape: (25724,)
y_test shape: (8575,)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4000      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6461 (25.24 KB)
Trainable params: 6461 (25.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5668 - accuracy: 0.7242
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7320
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5489 - accuracy: 0.7340
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7334
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7348
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5456 - accuracy: 0.7351
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5445 - accuracy: 0.7344
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7357
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7349
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5427 - accuracy: 0.7352

In [ ]:
# Evaluate the model on the test set
eval_result = nn.evaluate(X_test_scaled, y_test)

# Print the evaluation results
print(f"\nTest Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

268/268 [==============================] - 1s 2ms/step - loss: 0.5564 - accuracy: 0.7252

Test Loss: 0.5564315915107727, Test Accuracy: 0.7252478003501892


**First Optimisation try**

In [ ]:
# Adjust input data
nn_optimized = tf.keras.models.Sequential()
# First hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=128, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

nn_optimized.fit(X_train_scaled, y_train, epochs=100) #The number of epochs increased

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5649 - accuracy: 0.7236
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7318
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7334
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7335
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7348
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accuracy: 0.7360
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5431 - accuracy: 0.7362
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5430 - accuracy: 0.7363
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accura

In [ ]:
# Evaluate the optimized model on the test set
eval_result_optimized = nn_optimized.evaluate(X_train_scaled, y_train)

# Print the evaluation results
print(f"\nTest Loss: {eval_result_optimized[0]}, Test Accuracy: {eval_result_optimized[1]}")

804/804 [==============================] - 1s 1ms/step - loss: 0.5260 - accuracy: 0.7442

Test Loss: 0.5259720683097839, Test Accuracy: 0.7442077398300171


First try attempt result: Accuracy increased from 72% to 74%